# Ejecución de modelos RFSM con python


29/09/2021

**Salvador Navas Fernández** ([IH Cantabria](https://ihcantabria.com/), Santander, Spain)

En lo últimos años el avance de python y QGIS han permitido ir ganando terreno a ArcGIS cuya licencia no es libre. Por tanto en este trabajo se han realizado funciones que permiten realizar el trabajo en cualquier sistema operativo y de forma más optimizada y automática. El objetivo principal es el elaborar los inputs de partida del modelo RFSM y su ejecución, de esta forma se ha conseguido reducir lo tiempos de cómputo del proceso.

A continuación se muestra un ejemplo de principio a fin de cómo elaborar y proyecto de RFSM, su ejecución y la obtención de resultados.

In [1]:
from RFSM_python.RFSM_funtions import *
import warnings
warnings.filterwarnings('ignore')

# INPUT

En primer lugar es necesario configurar los nombres de los path donde se va a trabajar además de la configuración inicial del modelo RFSM.
Previamente se tiene que tener ya la carpeta configurada, es decir con la siguiente distribución de carpetas que se obtiene al ejecutar las siguientes líneas. 

```
project
└───ascii
└───bin
└───demos
└───doc
└───scripts
└───shp
└───src
└───test
```



In [2]:
path_project = '../tests/MURCIA/RFSM/' # Directorio donde se va a localizar el proyecto, es decir donde se va a simular el modelo
#COTA_ESTUDIO = 10
path_output = path_project+'/ascii/'
#CoastLine=path_project+'shp/CoastPolygon.shp'
DTM_CLIP_LC = '../tests/MURCIA/DATA_IN/MDT/mdt.asc'
path_install_RFSM = '../tests/IHRFSM_v1.0/'

In [3]:
#path_inputs_dinamicas='E:/RFSM/CASO_ASTURIAS/DINAMICAS_ASTURIAS/MARINE_DYNAMICS/'
n_periods = 13

In [4]:
TestID = 1
TestDesc = 'Murcia_Malla_1'
BCSetID = 1
StartTime = '0#h'
EndTime = str(n_periods)+'#h'
TimeStep = 1
SaveTimeStep = '20#m'
MaxTimeStep = 20
MinTimeStep = 0.0001
AlphaParameter = 0.7
ManningGlobalValue = 0.035
Results = '1'
make_dirs(path_install_RFSM,path_project,TestDesc,Results)

Una vez creadas las carpetas Lo más importante es que las carpetas contengan los ficheros necesarios, lo más importante es la carpeta ascii de donde partiremos para preprar el modelo. En esta carpeta se deben de encontrar los ficheros que se crean tras la ejecución de 'AccData'.

```
project
└───ascii
│   │   floodareas.asc
│   │   tblCell.csv
│   │   tblImpactZone.csv
│   │   tblIZNbrWidth.csv
│   │   tblIZNeighbour.csv
│   │   tblIZVolume.csv
│   │   tblParameters.csv
│   │   topography.asc
│   └───check
│       │   acccells_izid.csv
│       │   cellid.asc
│       │   dirdrainage.asc
│       │   izid1.asc
│       │   izid2.asc
│───bin
│───demos
│───doc
│───scripts
│───shp
│───src
│───test
```


# PROCESO

Una vez que se han obtenido la malla de proyecto de estudio, se procede a la ejecución del proceso para obtener los inputs necesarios del modelo RFSM

## IMPACT ZONES

In [5]:
Table_impact_zone, izcoast = impact_zones_process(path_project,DTM_CLIP_LC,cellsize=5,new_coast_Line = True)

C:\Anaconda3\envs\RFSM\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!({} != {'proj': 'utm', 'zone': 30, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)
C:\Anaconda3\envs\RFSM\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!({} != {'proj': 'utm', 'zone': 30, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)


180.58376002311707


Una vez que se ha ejecutado la línea anterior se crean dentro de la carpeta **shp** y **ascii** ficheros importantes para la configuración del modelo RFSM
```
project
└───ascii
│   │   izcoast.csv
│   │   tblImpactZone.csv
└───shp
│   │  izcoast.shp
```

El fichero **izcoast.shp** son las ImpactZones donde se incoporan las condiciones de contorno asociadas a *nivel* o *rebase*, pero en ocasiones cuando el corte del mdt no es limpio se generan celdas de contorno que no son correctas, por tanto es necesario modificar eliminando estas celdas del fichero **izcoast.shp**. Al nuevo fichero creado le llamamos **izcoast_2.shp** Para modificar la tabla de *izcoast* Se ejecuta la siguiente línea de código.

In [6]:
izcoast_modify(path_project,path_project+'shp/izcoast_2.shp', path_project+'ascii/izcoast.csv', path_project+'ascii/izcoast_def.csv')

C:\Anaconda3\envs\RFSM\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


## Puntos de entrada

In [128]:
Table_impact_zone=pd.read_csv(path_project+'ascii/tblImpactZone.csv',index_col=0)
izcoast=pd.read_csv(path_project+'ascii/izcoast.csv',index_col=0)

###  LEVEL IN A SERIES OF POINTS ALONG THE COAST

In [129]:
# Input 2: temporal
CF_FV = 1
xD = 569590.015
yD = 3110298.838
CM = 2.8
CI_pleamar = [1.6, 2, 2.4, 2.8, 2.4, 2, 1.6]
#CI_pleamar = [1.6, 2, 3,4, 3, 2, 1.6]

Tp = 22;
Hs = 8;
Dur = 10;

# Input 1: pleamar   


In [130]:
Table_impact_zone['IZRunOffCoef'] = 0.6
Table_impact_zone['RainfallZoneID'] = 1

BCTypeID_COAST = 2

In [131]:
tusrBCFlowLevel (path_project,TestDesc,Table_impact_zone,izcoast,BCSetID,BCTypeID_COAST,
                    CF_FV,xD,yD,CM,CI_pleamar, n_periods,input_=2)

100%|███████████████████████████████████████████████████████████████████████████████| 590/590 [00:03<00:00, 161.73it/s]


## Manning

In [132]:
create_file_manning(path_project,'F:/PIMA_ADAPTA_MURCIA/GIS/rugosidad',TestDesc,Table_impact_zone)

100%|███████████████████████████████████████████████████████████████████████████| 23983/23983 [03:25<00:00, 116.69it/s]


## Create XML

In [133]:
create_xml(TestID,TestDesc,BCSetID,StartTime,EndTime,TimeStep,SaveTimeStep,MaxTimeStep,MinTimeStep,AlphaParameter,ManningGlobalValue,path_project,Results)

## Ejecución 

In [134]:
execute_RFSM(path_project,TestDesc,TestID)

Assuming windows!


## Leer y exportar resultados

In [135]:
export_result_RFSM_2(path_project,TestDesc,Results)

100%|██████████████████████████████████████████████████████████████████████████| 23983/23983 [00:19<00:00, 1228.62it/s]
